# Spam Classification Training Notebook

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

In [2]:
# Cell 2: File paths
DATA_FILE = "spam.csv"
VECTOR_FILE = "vectorizer.pkl"
MODEL_DIR = "models/"

In [3]:
# Cell 3: Load dataset
def load_data(file_path):
    try:
        data = pd.read_csv(file_path, encoding="latin-1")
        data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
        data.rename(columns={'v1': 'class', 'v2': 'message'}, inplace=True)
        data['class'] = data['class'].map({'ham': 0, 'spam': 1})
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [4]:
# Cell 4: Preprocess data
def preprocess_data(data):
    if data.isnull().sum().any():
        print("Warning: Dataset contains null values. Dropping null entries.")
        data.dropna(inplace=True)
    X = data['message']
    y = data['class']
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_transformed = vectorizer.fit_transform(X)
    return X_transformed, y, vectorizer

In [5]:
# Cell 5: Train and evaluate models
def train_and_evaluate_model(model, model_name, x_train, x_test, y_train, y_test):
    print(f"Training {model_name}...")
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Results for {model_name}:")
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))
    print("Classification Report:\n", classification_report(y_test, predictions))
    print("-" * 60)
    return model, accuracy

In [6]:
# Cell 6: Save model or vectorizer
def save_pickle(obj, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(obj, f)

In [7]:
# Cell 7: Main script
import os

def main():
    # Load and preprocess data
    data = load_data(DATA_FILE)
    if data is None:
        return

    X, y, vectorizer = preprocess_data(data)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Save the vectorizer
    save_pickle(vectorizer, VECTOR_FILE)

    # Create model directory if it does not exist
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)

    # Models to train
    models = {
        "Naive_Bayes": MultinomialNB(),
        "SVM": SVC(kernel='linear', probability=True),
        "Decision_Tree": DecisionTreeClassifier(random_state=42),
        "Random_Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "Neural_Network": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
    }

    # Train and evaluate each model
    results = []
    for model_name, model in models.items():
        trained_model, accuracy = train_and_evaluate_model(model, model_name, x_train, x_test, y_train, y_test)
        results.append((model_name, accuracy))
        save_pickle(trained_model, f"{MODEL_DIR}{model_name}.pkl")

    # Summary of results
    print("\nModel Performance Summary:")
    for model_name, accuracy in results:
        print(f"{model_name}: {accuracy:.2f}")

if __name__ == "__main__":
    main()

Training Naive_Bayes...
Results for Naive_Bayes:
Accuracy: 0.9757847533632287
Confusion Matrix:
 [[965   0]
 [ 27 123]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.82      0.90       150

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.98      0.97      1115

------------------------------------------------------------
Training SVM...
Results for SVM:
Accuracy: 0.9757847533632287
Confusion Matrix:
 [[961   4]
 [ 23 127]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.97      0.85      0.90       150

    accuracy                           0.98      1115
   macro avg       0.97      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

-------------